[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/94_mapbox.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/94_mapbox.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Creating 3D maps with Mapbox**

In [ ]:
# %pip install -U leafmap

In [ ]:
import leafmap.mapbox as leafmap

You need a Mapbox access token to use the Mapbox widget. First, [sign up](https://account.mapbox.com/auth/signup/) for a free Mapbox account. Then, you can create a token by following the instructions [here](https://docs.mapbox.com/api/accounts/tokens/). Set `Mapbox_TOKEN` as an environment variable to use the Mapbox widget. Alternatively, uncomment the following code block and replace `YOUR-API-TOKEN` with your Mapbox access token.

In [ ]:
# import os
# os.environ['MAPBOX_TOKEN'] = 'YOUR-API-TOKEN'

In [ ]:
m = leafmap.Map(center=[-100, 40], zoom=1.2, height="600px")
m

![](https://i.imgur.com/1PbtnQE.gif)

In [ ]:
m = leafmap.Map(style="mapbox://styles/mapbox/satellite-streets-v12")
m

In [ ]:
m.clicked_lnglat

In [ ]:
m.zoom

In [ ]:
m.center

In [ ]:
m.bounds

In [ ]:
esm = """

    const map = new mapboxgl.Map({
        container: 'map',
        zoom: 14,
        center: [-114.26608, 32.7213],
        pitch: 80,
        bearing: 41,
        // Choose from Mapbox's core styles, or make your own style with Mapbox Studio
        style: 'mapbox://styles/mapbox/satellite-streets-v12'
    });

    map.on('style.load', () => {
        map.addSource('mapbox-dem', {
            'type': 'raster-dem',
            'url': 'mapbox://mapbox.mapbox-terrain-dem-v1',
            'tileSize': 512,
            'maxzoom': 14
        });
        // add the DEM source as a terrain layer with exaggerated height
        map.setTerrain({ 'source': 'mapbox-dem', 'exaggeration': 1.5 });
    });


"""

In [ ]:
m.set_esm(esm)
m

![](https://i.imgur.com/xvdrvpx.png)

In [ ]:
esm = """

    (async () => {
        const map = new mapboxgl.Map({
            container: 'map',
            zoom: 13,
            center: [6.6301, 45.35625],
            pitch: 80,
            bearing: 160,
            interactive: false,
            // Choose from Mapbox's core styles, or make your own style with Mapbox Studio
            style: 'mapbox://styles/mapbox/satellite-v9'
        });

        await map.once('style.load');

        // Add daytime fog
        map.setFog({
            'range': [-1, 2],
            'horizon-blend': 0.3,
            'color': 'white',
            'high-color': '#add8e6',
            'space-color': '#d8f2ff',
            'star-intensity': 0.0
        });

        // Add 3D terrain
        map.addSource('mapbox-dem', {
            'type': 'raster-dem',
            'url': 'mapbox://mapbox.terrain-rgb',
            'tileSize': 512,
            'maxzoom': 14
        });
        map.setTerrain({
            'source': 'mapbox-dem',
            'exaggeration': 1.5
        });

        // Run a timing loop to switch between day and night
        await map.once('idle');

        let lastTime = 0.0;
        let animationTime = 0.0;
        let cycleTime = 0.0;
        let night = true;

        const initialBearing = map.getBearing();

        function frame(time) {
            const elapsedTime = (time - lastTime) / 1000.0;

            animationTime += elapsedTime;
            cycleTime += elapsedTime;

            if (cycleTime >= 10.0) {
                if (night) {
                    // night fog styling
                    map.setFog({
                        'range': [-1, 2],
                        'horizon-blend': 0.3,
                        'color': '#242B4B',
                        'high-color': '#161B36',
                        'space-color': '#0B1026',
                        'star-intensity': 0.8
                    });
                } else {
                    // day fog styling
                    map.setFog({
                        'range': [-1, 2],
                        'horizon-blend': 0.3,
                        'color': 'white',
                        'high-color': '#add8e6',
                        'space-color': '#d8f2ff',
                        'star-intensity': 0.0
                    });
                }

                night = !night;
                cycleTime = 0.0;
            }

            const rotation = initialBearing + animationTime * 2.0;
            map.setBearing(rotation % 360);

            lastTime = time;

            window.requestAnimationFrame(frame);
        }

        window.requestAnimationFrame(frame);
    })();

"""

In [ ]:
m.set_esm(esm)
m

![](https://i.imgur.com/ZRRUK3v.png)

In [ ]:
m = leafmap.Map(style="mapbox://styles/mapbox/satellite-streets-v12")
m

In [ ]:
# import ee
# ee.Authenticate()
# ee.Initialize(project="YOUR-PROJECT-ID")

In [ ]:
# dataset = ee.ImageCollection("ESA/WorldCover/v200").first()
# vis_params = {"bands": ["Map"]}
# url = leafmap.ee_tile_url(dataset, vis_params)

In [ ]:
esm = """
    const map = new mapboxgl.Map({
        container: 'map',
        zoom: 1.2,
        center: [-100, 40],
        // Choose from Mapbox's core styles, or make your own style with Mapbox Studio
        style: 'mapbox://styles/mapbox/satellite-streets-v12'
    });

    map.on('style.load', () => {
        map.addSource('mapbox-dem', {
            'type': 'raster-dem',
            'url': 'mapbox://mapbox.mapbox-terrain-dem-v1',
            'tileSize': 512,
            'maxzoom': 14
        });
        // add the DEM source as a terrain layer with exaggerated height
        map.setTerrain({ 'source': 'mapbox-dem', 'exaggeration': 1.5 });

        map.addSource('raster-tiles', {
            'type': 'raster',
            'tiles': [
                'https://THE-TILE-URL' // replace with the actual tile URL
            ],
            'tileSize': 256
        });

        map.addLayer({
            'id': 'raster-layer',
            'type': 'raster',
            'source': 'raster-tiles',
            'paint': {
                'raster-opacity': 0.7 // Adjust the opacity as needed
            }
        });
        
    });

"""

In [ ]:
m.set_esm(esm)
m

![](https://i.imgur.com/8YXh4Cv.png)